In [1]:
import sys
import nltk
import sklearn
import numpy
import pandas


## Load the dataset


In [2]:
import pandas as pd
import numpy as np

import os
print(os.getcwd())
os.listdir(os.getcwd())

df = pd.read_csv('SMSSpamCollection', header=None, encoding='utf-8',sep='\t')

C:\Users\Srishti Kotnis


In [3]:
print(df.info())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
0    5572 non-null object
1    5572 non-null object
dtypes: object(2)
memory usage: 87.1+ KB
None


In [4]:
print(df.head())

      0                                                  1
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...


In [5]:
#check class distribution

classes = df[0]
print(classes.value_counts())

ham     4825
spam     747
Name: 0, dtype: int64


## 2. Preprocess the data


In [20]:
#convert class labels to binary values, 0=ham, 1=spam

from sklearn.preprocessing import LabelEncoder

encoder=LabelEncoder()
#classes --first column of the dataframe
y=encoder.fit_transform(classes)

print(classes.iloc[1:11])
print(y[1:11])

1      ham
2     spam
3      ham
4      ham
5     spam
6      ham
7      ham
8     spam
9     spam
10     ham
Name: 0, dtype: object
[0 1 0 0 1 0 0 1 1 0]


In [21]:
#store the sms data
text_msg=df[1]
print(text_msg[1:10])

1                        Ok lar... Joking wif u oni...
2    Free entry in 2 a wkly comp to win FA Cup fina...
3    U dun say so early hor... U c already then say...
4    Nah I don't think he goes to usf, he lives aro...
5    FreeMsg Hey there darling it's been 3 week's n...
6    Even my brother is not like to speak with me. ...
7    As per your request 'Melle Melle (Oru Minnamin...
8    WINNER!! As a valued network customer you have...
9    Had your mobile 11 months or more? U R entitle...
Name: 1, dtype: object


In [22]:
#use regulsr expression  to replace email address,urls

#replace email addesees with 'emailaddr'
processed = text_msg.str.replace(r'^.+@[^\.].*\.[a-z]{2,}$','emailadd')


#replace urls with 'webaddress'
processed = processed.str.replace(r'^http://[a-zA-Z0-9\-\.]+\.[a-zA-Z]{2,3}(/\S*)?$','webaddress')

#replace money sysmbols with 'moneysymb'
processed =processed.str.replace(r'£|\$','moneysymb')

#replace 10digit ph no. with 'phonenumber'
processed = processed.str.replace(r'^\(?[\d]{3}\)?[s-]?[\d]{3}[\s-]?[\d]{4}$','phonenumber')

#replace normal no. with numbr
processed = processed.str.replace(r'\d+(\.\d+)?','numbr')




In [23]:
#remove punctuation
processed = processed.str.replace(r'[^\w\d\s]',' ')

#replace whitespace
processed=processed.str.replace(r'\s+',' ')

#replace leading and trailing whitespace
processed=processed.str.replace(r'^\s+|\s?$','')




In [25]:
#change to lowercase
processed= processed.str.lower()
print(processed)


0       go until jurong point crazy available only in ...
1                                 ok lar joking wif u oni
2       free entry in numbr a wkly comp to win fa cup ...
3             u dun say so early hor u c already then say
4       nah i don t think he goes to usf he lives arou...
5       freemsg hey there darling it s been numbr week...
6       even my brother is not like to speak with me t...
7       as per your request melle melle oru minnaminun...
8       winner as a valued network customer you have b...
9       had your mobile numbr months or more u r entit...
10      i m gonna be home soon and i don t want to tal...
11      six chances to win cash from numbr to numbr nu...
12      urgent you have won a numbr week free membersh...
13      i ve been searching for the right words to tha...
14                      i have a date on sunday with will
15      xxxmobilemovieclub to use your credit click th...
16                                 oh k i m watching here
17      eh u r

In [26]:
#remove stop words
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))
processed = processed.apply(lambda x:' '.join(term for term in x.split() if term not in stop_words))

[nltk_data] Downloading package stopwords to C:\Users\Srishti
[nltk_data]     Kotnis\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [27]:
from nltk.stem import PorterStemmer
ps= PorterStemmer()

processed = processed.apply(lambda x: ' '.join(ps.stem(term) for term in x.split()))


In [28]:
print (processed)


0       go jurong point crazi avail bugi n great world...
1                                   ok lar joke wif u oni
2       free entri numbr wkli comp win fa cup final tk...
3                     u dun say earli hor u c alreadi say
4                    nah think goe usf live around though
5       freemsg hey darl numbr week word back like fun...
6           even brother like speak treat like aid patent
7       per request mell mell oru minnaminungint nurun...
8       winner valu network custom select receivea mon...
9       mobil numbr month u r entitl updat latest colo...
10      gonna home soon want talk stuff anymor tonight...
11      six chanc win cash numbr numbr numbr pound txt...
12      urgent numbr week free membership moneysymbnum...
13      search right word thank breather promis wont t...
14                                            date sunday
15      xxxmobilemovieclub use credit click wap link n...
16                                             oh k watch
17      eh u r

In [29]:
#nltk.download('punkt')

from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
all_words = []

for message in processed:
    words = word_tokenize(message)
    for w in words:
        all_words.append(w)
        
         
all_words = nltk.FreqDist(all_words)

In [30]:
print('Number of words {}'.format(len(all_words)))
print('Most common words: {}'.format(all_words.most_common(15)))


Number of words 6579
Most common words: [('numbr', 2648), ('u', 1207), ('call', 674), ('go', 456), ('get', 451), ('ur', 391), ('gt', 318), ('lt', 316), ('come', 304), ('moneysymbnumbr', 303), ('ok', 293), ('free', 284), ('day', 276), ('know', 275), ('love', 266)]


In [31]:
 #use 1500 most common words as features
word_features=list(all_words.keys())[:1500]



In [32]:

from nltk.tokenize import word_tokenize

 #use 1500 most common words as features
word_features=list(all_words.keys())[:1500]
def find_features(message):
    words = word_tokenize(message)
    features = {}
    for word in word_features:
        features = {}
        features[word] = (word in words)
    
    return (features)





In [33]:

#find features for all messages
messages = []
for i in range(0,len(processed)):
    messages.append((processed[i],y[i]))

In [34]:
messages[:10]

[('go jurong point crazi avail bugi n great world la e buffet cine got amor wat',
  0),
 ('ok lar joke wif u oni', 0),
 ('free entri numbr wkli comp win fa cup final tkt numbrst may numbr text fa numbr receiv entri question std txt rate c appli numbrovernumbr',
  1),
 ('u dun say earli hor u c alreadi say', 0),
 ('nah think goe usf live around though', 0),
 ('freemsg hey darl numbr week word back like fun still tb ok xxx std chg send moneysymbnumbr rcv',
  1),
 ('even brother like speak treat like aid patent', 0),
 ('per request mell mell oru minnaminungint nurungu vettam set callertun caller press numbr copi friend callertun',
  0),
 ('winner valu network custom select receivea moneysymbnumbr prize reward claim call numbr claim code klnumbr valid numbr hour',
  1),
 ('mobil numbr month u r entitl updat latest colour mobil camera free call mobil updat co free numbr',
  1)]

In [35]:
feature_set = [[find_features(message),category] for (message, category) in messages]

In [36]:
from random import shuffle 
shuffle(feature_set)

In [37]:
print(len(feature_set))


5572


In [40]:
from sklearn import model_selection

training, testing = model_selection.train_test_split(feature_set, test_size=0.25,random_state=0)

In [41]:
print('Training:{}'.format(len(training)))
print('Testing:{}'.format(len(testing)))

Training:4179
Testing:1393


## 4. scikit learn classifiers with nltk

In [42]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix

In [45]:
#Define Models to train
names = ["K Nearest Neighbors", "Decision Tree", "Random Forest", "Logistic Regression", "SGD Classifier",
         "Naive Bayes", "SVM Linear"]

classifiers = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    LogisticRegression(),
    SGDClassifier(max_iter=100),
    MultinomialNB(),
    SVC(kernel = 'linear')
]

models = []
for i in range(0,len(classifiers)):
    models.append((names[i],classifiers[i]))

print(models)

[('K Nearest Neighbors', KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')), ('Decision Tree', DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')), ('Random Forest', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))

In [44]:
from nltk.classify.scikitlearn import SklearnClassifier

for (name, model) in models:
    nltk_model = SklearnClassifier(model)
    nltk_model.train(training)
    accuracy = nltk.classify.accuracy(nltk_model,testing)*100
    print(f'{name} :  Accuracy \t:    {accuracy}')

K Nearest Neighbors :  Accuracy 	:    86.86288585786073
Decision Tree :  Accuracy 	:    86.86288585786073
Random Forest :  Accuracy 	:    86.86288585786073
Logistic Regression :  Accuracy 	:    86.86288585786073
SGD Classifier :  Accuracy 	:    86.86288585786073
Naive Bayes :  Accuracy 	:    86.86288585786073
SVM Linear :  Accuracy 	:    86.86288585786073


In [46]:
# Ensemble methods - Voting classifier
from sklearn.ensemble import VotingClassifier

names = ["K Nearest Neighbors", "Decision Tree", "Random Forest", "Logistic Regression", "SGD Classifier",
         "Naive Bayes", "SVM Linear"]

classifiers = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    LogisticRegression(),
    SGDClassifier(max_iter = 100),
    MultinomialNB(),
    SVC(kernel = 'linear')
]

models =[]
for i in range(0,len(classifiers)):
    models.append((names[i],classifiers[i]))
nltk_ensemble = SklearnClassifier(VotingClassifier(estimators = models, voting = 'hard', n_jobs = -1))
nltk_ensemble.train(training)
accuracy = nltk.classify.accuracy(nltk_model, testing)*100
print("Voting Classifier: Accuracy: {}".format(accuracy))

Voting Classifier: Accuracy: 86.86288585786073


In [47]:
#make class label prediction for testing set
txt_features,labels= zip(*testing)

prediction= nltk_ensemble.classify_many(txt_features)

C:\Users\SrishtiKotnis\MyDocuments\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [48]:
print(classification_report(labels,prediction))
pd.DataFrame(
    confusion_matrix(labels,prediction),
    index=[['actual','actual'],['ham','spam']],
    columns=[['predicted','predicted'],['ham','spam']])

             precision    recall  f1-score   support

          0       0.87      1.00      0.93      1210
          1       0.00      0.00      0.00       183

avg / total       0.75      0.87      0.81      1393



C:\Users\SrishtiKotnis\MyDocuments\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


predicted     
                  ham spam
actual ham       1210    0
       spam       183    0